In [1]:
import os
import tarfile
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler

In [2]:
tar_folder = "Images"          #Folder with the TAR files
pca_output = "Images_PCA.parquet"  #Final file with PCA
n_components = 100                   #PCA number of components. More number, less compression, better quality
batch_size = 1000                     #Set of images to process in memory

scaler = StandardScaler() #PCA depends on variance: if there is one characteristic with bigger values,
                          #it would dominate. In order to prevent that, we scalate so everything would contribute the same

ipca = IncrementalPCA(n_components=n_components, batch_size=batch_size) #Inicializes an incremental PCA.
# It allows processing large volumes of data in batches, without loading everything into memory at once.

Se encontraron 70 archivos TAR.


In [3]:
#FIRST PASS of preprocessing before applying PCA: Calculate the mean and global standard devitation of all pixels

X_batch = [] #Creates an empty list to temporarily store pixel vectors for each batch.
for tar_path in tar_files:
    with tarfile.open(tar_path, "r") as tar:
        members = [m for m in tar.getmembers() if m.name.lower().endswith(".jpg")] #Filters only the .jpg images inside the archive.
        for member in members:
            f = tar.extractfile(member) #Extracts the current image file as a binary object in memory.
            if f is not None: #Checking if the image is successfully extracted.
                try:
                    img = Image.open(BytesIO(f.read())).convert("RGB").resize((64, 64)) #Reads the image bytes into memory, opens it,
                                                                                        #ensures 3 color channels and resizes to a fixed shape
                    pixels = np.array(img).astype(np.float32).flatten() / 255.0 #Converts the image into a NumPy array, flattens it from
                                                                                #64×64×3=12288-length vector and normalizes pixel values to
                                                                                # the range [0, 1].
                                                                                #It makes values float32 (needed by scaler/IPCA).
                    X_batch.append(pixels) #Adds the pixel vector to the current batch.
                    
                    if len(X_batch) >= batch_size:
                        scaler.partial_fit(np.array(X_batch))  #When the batch is full, scaler.partial_fit() updates the mean and standard
                                                                #deviation for scaling using just that batch.

                        X_batch = [] #It clears X_batch to start collecting a new batch.
                except Exception as e:
                    print(f"Error reading {member.name}: {e}")

#Processing the last set
if X_batch:
    scaler.partial_fit(np.array(X_batch))
    X_batch = []

Calculando escalador incremental...


In [4]:
#SECOND PASS adjusting incremental PCA in sets: scalates all the data and trains IncrementalPCA

X_batch = []
for tar_path in tar_files:
    with tarfile.open(tar_path, "r") as tar:
        members = [m for m in tar.getmembers() if m.name.lower().endswith(".jpg")]
        for member in members:
            f = tar.extractfile(member)
            if f is not None:
                try:
                    img = Image.open(BytesIO(f.read())).convert("RGB").resize((64, 64))
                    pixels = np.array(img).astype(np.float32).flatten() / 255.0
                    X_batch.append(pixels)
                    
                    if len(X_batch) >= batch_size:
                        X_scaled = scaler.transform(np.array(X_batch)) #Convert the list to a NumPy array of shape
                                                                       #(n_samples_in_batch, n_features) and standardize it using the
                                                                       #scaler fitted during the first pass. This centers each feature
                                                                       #to mean 0 and scales to unit variance
                                                                       #(using global mean/std learned earlier).
                        ipca.partial_fit(X_scaled) #Feed the standardized batch to the IncrementalPCA. partial_fit() updates the PCA internal
                                                   #state (estimates of components, means...) incrementally. Each call refines the global PCA
                                                   #using the new batch.
                        X_batch = []
                except:
                    pass

#Last set
if X_batch:
    X_scaled = scaler.transform(np.array(X_batch))
    ipca.partial_fit(X_scaled)
    X_batch = []

Ajustando PCA incremental...


In [ ]:
#These two codes can't merge (even if they seem to be very similar)

#When using "StandardScaler", it needs to know the global mean and global standard deviation of the entire dataset
#in order to scale the data correctly. The problem is that during the first pass, those statistics are not yet known.
#They are still being computed gradually. If "scaler.transform()" is applied at the same time as "scaler.partial_fit()",
#you would be scaling using partial and changing values, not the final mean and standard deviation.
#This would cause each batch to be scaled differently and if the data aren’t scaled consistently, the PCA would learn
#distorted or incorrect components.

In [5]:
#THIRD PASS transforming and saving PCA

all_filenames = []
all_pca = []
X_batch = []

for tar_path in tar_files:
    with tarfile.open(tar_path, "r") as tar:
        members = [m for m in tar.getmembers() if m.name.lower().endswith(".jpg")]
        for member in members:
            f = tar.extractfile(member)
            if f is not None:
                try:
                    img = Image.open(BytesIO(f.read())).convert("RGB").resize((64, 64))
                    pixels = np.array(img).astype(np.float32).flatten() / 255.0
                    X_batch.append(pixels)
                    #Clean name: just base name without extention
                    clean_name = os.path.splitext(os.path.basename(member.name))[0] #Create a “clean” identifier
                                                                                    #(filename without directory or extension) for the image.
                    all_filenames.append(clean_name) #Append that clean name to the global filename list immediately
                    
                    if len(X_batch) >= batch_size:
                        X_scaled = scaler.transform(np.array(X_batch)) #Use the already-fitted StandardScaler (from FIRST PASS)
                                                                       #to standardize the batch.
                        pcs = ipca.transform(X_scaled) #Use the already-fitted IncrementalPCA (from SECOND PASS) to project
                                                       #the standardized batch into PCA
                        all_pca.append(pcs)
                        X_batch = []
                except:
                    pass

#Last set
if X_batch:
    X_scaled = scaler.transform(np.array(X_batch))
    pcs = ipca.transform(X_scaled)
    all_pca.append(pcs)

Transformando imágenes y guardando PCA...


In [6]:
#Merging all the sets
X_pca = np.vstack(all_pca)
pca_columns = [f"PC{i+1}" for i in range(n_components)]
df_pca = pd.DataFrame(X_pca, columns=pca_columns)
df_pca.insert(0, "filename", all_filenames)

In [7]:
# Save final PCA
df_pca.to_parquet(pca_output, index=False)

PCA completado y guardado en: Images_PCA.parquet
Total de imágenes procesadas: 69352
Varianza explicada total: 89.16%


In [9]:
#Read CSV and PCA
df_catalog = pd.read_csv("ZooSpecPhotoDR19_filtered.csv")
df_pca = pd.read_parquet("Images_PCA.parquet")

#Convert both to string so we can compare the 'objid' columns with the names of the images (saved as filename in the PCA)
df_catalog["objid"] = df_catalog["objid"].astype(str)
df_pca["filename"] = df_pca["filename"].astype(str)

# Merge
df_final = pd.merge(df_catalog, df_pca, left_on="objid", right_on="filename", how="inner")

#Eliminate the 'filename' column (duplicated)
df_final = df_final.drop(columns=["filename"])

#Save combinated dataset
df_final.to_parquet("Dataset_combinado.parquet", index=False)

   Unnamed: 0            specobjid                objid            dr7objid  \
0           1  1578598304118237184  1237661463301456237  587735742076551517   
1           3  1578599678507771904  1237661463301521650  587735742076616950   
2           7  1578583460711262208  1237661463301718141  587735742076813455   
3          20  1578572190717077504  1237661463302045898  587735742077141189   
4          29  1579718437544945664  1237661463302308093  587735742077403410   

         ra       dec  p_el_debiased  p_cs_debiased  spiral  elliptical  ...  \
0  233.7615  34.60428          0.000          1.000       1           0  ...   
1  233.9483  34.48045          0.069          0.931       1           0  ...   
2  234.3422  34.38433          0.015          0.985       1           0  ...   
3  235.0977  33.95142          0.040          0.939       1           0  ...   
4  235.8138  33.56461          0.000          1.000       1           0  ...   

       PC91      PC92      PC93      PC94   